In [1]:
import torch
from torch_geometric.data import InMemoryDataset, download_url, Data, Batch
from torch import nn
from torch.nn import functional as F
import os
import pandas as pd
import numpy as np
import pickle
import itertools
import jax
from jax import numpy as jnp
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
import mendeleev
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler, HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import os

In [2]:
import os
from torch.utils.data import TensorDataset, DataLoader

In [3]:
files_train = os.listdir("/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/")

In [4]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/"
files_train = os.listdir(directory)
for file in np.sort(files_train):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [5]:
y_train = pd.concat(labels)
X_train = pd.concat(data)

In [6]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/test/"
files_test = os.listdir(directory)
for file in np.sort(files_test):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [7]:
X_temp_train = X_train[(y_train[0] == 2).to_numpy()]
y_temp_train = y_train[(y_train[0] == 2).to_numpy()]

In [8]:
y_test = pd.concat(labels)
X_test = pd.concat(data)

In [9]:
X_temp_test = X_test[(y_test[0] == 2).to_numpy()]
y_temp_test = y_test[(y_test[0] == 2).to_numpy()]

In [10]:
train_dataset = TensorDataset(torch.Tensor(X_temp_train.to_numpy()),torch.Tensor(y_temp_train.to_numpy())) # create your datset
train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=4096, shuffle=True, drop_last=True)
test_dataset = TensorDataset(torch.Tensor(X_temp_test.to_numpy()),torch.Tensor(y_temp_test.to_numpy())) # create your datset
test_dataloader = DataLoader(test_dataset, num_workers=12, batch_size=4096, shuffle=True)

In [11]:
del X_train, X_test, data, labels

In [12]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error

In [13]:
# reg = RandomForestRegressor(n_estimators=8, max_depth = 16)

In [14]:
# reg.fit(X_temp_train,y_temp_train)
# y_pred = reg.predict(X_temp_test)
# print(mean_absolute_error(y_pred, y_temp_test))

In [15]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class ResNetGated(nn.Module):
    def __init__(self, init_dim, hidden_dim, layers, p_dropout):
        super().__init__()
        self.p_dropout = p_dropout
        assert layers > 0
        self.layers = nn.ModuleList([nn.Sequential(nn.Linear(init_dim, hidden_dim),
                             nn.ReLU(),
                             nn.BatchNorm1d(hidden_dim),
                             nn.Linear( hidden_dim, init_dim)) for i in range(layers)])
        self.gates = nn.Parameter(torch.Tensor(layers))
        self.layers.apply(init_weights)
    def forward(self, x):
        range_gates = torch.sigmoid(self.gates)
        for i, layer in enumerate(self.layers):
            x = F.relu(x)
            x = (range_gates[i])*layer(x) + (1-range_gates[i])*x
        return x

    
class Net(torch.nn.Module):
    def __init__(self, init_dim, hidden_dim, hidden_width, layers, p_dropout):
        super().__init__()
        self.dr = nn.Sequential(nn.Linear(init_dim, 512),
                               nn.ReLU(),
                               nn.Dropout(p=p_dropout),
                               nn.Linear(512, hidden_dim))
        # self.ad = nn.Linear(init_dim, hidden_dim)
        self.fc = ResNetGated(hidden_dim, hidden_width, layers, p_dropout)
        self.reg = nn.Linear(hidden_dim, 1)
        self.fc.apply(init_weights)
        self.reg.apply(init_weights)
        self.dr.apply(init_weights)
    def forward(self, x):
        x = self.dr(x)
        x = self.fc(x)
        x = self.reg(x)
        return x

In [16]:
def train(model, device, data ,loss_fn, optimizer):
    model.train()
    train_losses = []
    optimizer.zero_grad()
    for i, batch in enumerate(data):
        data, labels = batch[0].to(device), batch[1][:, 1].to(device)
        logits = model(data)
        loss = loss_fn()
        output=loss(logits.squeeze(), labels.squeeze())
        output.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
        optimizer.step()
        train_loss = output.data.cpu().numpy()
        train_losses.append(train_loss)    
    return np.mean(train_losses)

### Testing function
def test(model, device, data, loss_fn):
    # Set evaluation mode for encoder and decoder
    model.eval()
    test_losses = []
    with torch.no_grad(): # No need to track the gradients
        for i, batch in enumerate(data):
            data, labels = batch[0].to(device), batch[1][:, 1].to(device)
            logits = model(data)
            loss = loss_fn()
            output=loss(logits.squeeze(), labels.squeeze())
            test_loss = output.data.cpu().numpy()
            test_losses.append(test_loss)  
    return np.mean(test_losses)

In [18]:
### Define the loss function
loss_fn = nn.MSELoss

lr= 0.0001
weight_decay = 0.00001
hidden_dim = 256
hidden_width=4096
layers=2
p_dropout = 0.05
### Set the random seed for reproducible results
torch.manual_seed(0)

model = Net(776, hidden_dim, hidden_width, layers, p_dropout=p_dropout)
params_to_optimize = [
    {'params': model.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=weight_decay)
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
model.to(device)

Selected device: cuda


Net(
  (dr): Sequential(
    (0): Linear(in_features=776, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
  )
  (fc): ResNetGated(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
    )
  )
  (reg): Linear(in_features=256, out_features=1, bias=True)
)

In [19]:
import datetime
date = datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

RUN = date

In [20]:
best_loss = 1000
num_epochs = 100
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
for epoch in range(num_epochs):
    if epoch == 0:
        for par in optim.param_groups:
            par["lr"] = 0.0001
    elif epoch< 10:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * 1.2
    else:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * decay
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/100 	 train loss 1665.3095703125 	 	 val loss 1820.220703125

 EPOCH 2/100 	 train loss 346.9549865722656 	 	 val loss 338.9229431152344

 EPOCH 3/100 	 train loss 30.08403205871582 	 	 val loss 53.6082763671875

 EPOCH 4/100 	 train loss 15.808185577392578 	 	 val loss 48.82148361206055

 EPOCH 5/100 	 train loss 8.39289665222168 	 	 val loss 12.222000122070312

 EPOCH 6/100 	 train loss 15.520566940307617 	 	 val loss 7.532541751861572

 EPOCH 7/100 	 train loss 13.10477352142334 	 	 val loss 19.154733657836914

 EPOCH 8/100 	 train loss 16.08954429626465 	 	 val loss 4.652970790863037

 EPOCH 9/100 	 train loss 10.119866371154785 	 	 val loss 5.694546222686768

 EPOCH 10/100 	 train loss 11.137431144714355 	 	 val loss 39.36540603637695

 EPOCH 11/100 	 train loss 7.430952548980713 	 	 val loss 7.5882039070129395

 EPOCH 12/100 	 train loss 16.738107681274414 	 	 val loss 23.636734008789062

 EPOCH 13/100 	 train loss 13.325322151184082 	 	 val loss 3.088170051574707

 EPO

In [21]:
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
num_epochs = 200
size = 4096
for epoch in range(num_epochs):
    if epoch%50 == 0:
        for par in optim.param_groups:
            par["weight_decay"] = par["weight_decay"]/2
            size = size * 2
            train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/200 	 train loss 1.2265324592590332 	 	 val loss 2.0221176147460938

 EPOCH 2/200 	 train loss 0.9032110571861267 	 	 val loss 1.1229254007339478

 EPOCH 3/200 	 train loss 0.7776583433151245 	 	 val loss 1.2039812803268433

 EPOCH 4/200 	 train loss 0.7785587906837463 	 	 val loss 1.081817626953125

 EPOCH 5/200 	 train loss 0.9297778606414795 	 	 val loss 1.1209232807159424

 EPOCH 6/200 	 train loss 0.7431045770645142 	 	 val loss 2.223788022994995

 EPOCH 7/200 	 train loss 1.0136630535125732 	 	 val loss 1.6535558700561523

 EPOCH 8/200 	 train loss 0.85408616065979 	 	 val loss 1.4740031957626343

 EPOCH 9/200 	 train loss 0.7317317128181458 	 	 val loss 1.1597908735275269

 EPOCH 10/200 	 train loss 0.8134658932685852 	 	 val loss 2.081273317337036

 EPOCH 11/200 	 train loss 1.107327938079834 	 	 val loss 1.3364814519882202

 EPOCH 12/200 	 train loss 0.9919090270996094 	 	 val loss 1.1478272676467896

 EPOCH 13/200 	 train loss 0.7327749729156494 	 	 val loss 1.62688

In [ ]:
decay = 0.98
num_epochs = 500
size = 4096
for epoch in range(num_epochs):
    # if epoch%50 == 0:
    #     for par in optim.param_groups:
    #         par["weight_decay"] = par["weight_decay"]/2
    #         size = size * 2
    #         train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))

In [ ]:
print(RUN)